In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow.compat.v2 as tf
from sklearn import preprocessing
tf.enable_v2_behavior()

import tensorflow_probability as tfp

sns.reset_defaults()
sns.set_context(context='talk',font_scale=0.7)

tfd = tfp.distributions

from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if tf.test.gpu_device_name() != '/device:GPU:0':
  print('WARNING: GPU device not found.')
else:
  print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

SUCCESS: Found GPU: /device:GPU:0


In [ ]:
data = pd.read_csv('drive/MyDrive/cleaned_data.csv')

In [ ]:
masses = pd.read_csv('drive/MyDrive/rcsed_8_logMstar_gal.csv')

In [ ]:
data.shape

(4109726, 86)

In [ ]:
mask = ~masses['logMstar_gal'].isna()
y = masses[mask]
y = y.reset_index()
y = y.drop(columns=['index'])

In [ ]:
data = data.drop(columns=['Unnamed: 0'])

In [ ]:
X = data[mask]
X = X.reset_index()
X = X.drop(columns=['index'])

In [ ]:
X = X.fillna(X.mean())

In [ ]:
#x_scaled = preprocessing.RobustScaler().fit_transform(X)
#X1=pd.DataFrame(x_scaled, columns=X.columns)

In [ ]:
#X1.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
scal = preprocessing.RobustScaler()

In [ ]:
X_train = scal.fit_transform(X_train)
X_test = scal.transform(X_test)

In [ ]:
negloglik = lambda y, p_y: -p_y.log_prob(y)

In [ ]:
def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
  n = kernel_size + bias_size
  c = np.log(np.expm1(1.))
  return tf.keras.Sequential([
      tfp.layers.VariableLayer(2 * n, dtype=dtype),
      tfp.layers.DistributionLambda(lambda t: tfd.Independent(
          tfd.Normal(loc=t[..., :n],
                     scale=1e-5 + tf.nn.softplus(c + t[..., n:])),
          reinterpreted_batch_ndims=1)),
  ])


def prior_trainable(kernel_size, bias_size=0, dtype=None):
  n = kernel_size + bias_size
  return tf.keras.Sequential([
      tfp.layers.VariableLayer(n, dtype=dtype),
      tfp.layers.DistributionLambda(lambda t: tfd.Independent(
          tfd.Normal(loc=t, scale=1),
          reinterpreted_batch_ndims=1)),
  ])

**Model 5**

In [ ]:
class RBFKernelFn(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super(RBFKernelFn, self).__init__(**kwargs)
    dtype = kwargs.get('dtype', None)

    self._amplitude = self.add_variable(
            initializer=tf.constant_initializer(0),
            dtype=dtype,
            name='amplitude')
    
    self._length_scale = self.add_variable(
            initializer=tf.constant_initializer(0),
            dtype=dtype,
            name='length_scale')

  def call(self, x):
    # Never called -- this is just a layer so it can hold variables
    # in a way Keras understands.
    return x

  @property
  def kernel(self):
    return tfp.math.psd_kernels.ExponentiatedQuadratic(
      amplitude=tf.nn.softplus(0.1 * self._amplitude),
      length_scale=tf.nn.softplus(5. * self._length_scale)
    )

In [ ]:
# For numeric stability, set the default floating-point dtype to float64
tf.keras.backend.set_floatx('float64')
# Build model.
num_inducing_points = 500
model5 = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=[85]),
    tf.keras.layers.Dense(32, kernel_initializer='ones', use_bias=False, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(64, kernel_initializer='ones', use_bias=False, activation='relu'),
    tf.keras.layers.Dropout(0.25),

    tfp.layers.VariationalGaussianProcess(
        num_inducing_points=num_inducing_points,
        kernel_provider=RBFKernelFn(),
        event_shape=[1],
        unconstrained_observation_noise_variance_initializer=(
            tf.constant_initializer(np.log(np.expm1(1.)).astype('float64'))),
    ),
])




# Do inference.
batch_size = 128
loss = lambda y, rv_y: rv_y.variational_loss(
    y, kl_weight=np.array(batch_size, 'float64') / X.shape[0])
model5.compile(optimizer=tf.optimizers.Adam(learning_rate=0.005), loss=loss)

history = model5.fit(X_train, y_train, batch_size=batch_size, epochs=100, verbose=2)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.


/usr/local/lib/python3.7/dist-packages/tensorflow_probability/python/distributions/gaussian_process.py:312: UserWarning: Unable to detect statically whether the number of index_points is 1. As a result, defaulting to treating the marginal GP at `index_points` as a multivariate Gaussian. This makes some methods, like `cdf` unavailable.
  'Unable to detect statically whether the number of index_points is '


Epoch 1/100


/usr/local/lib/python3.7/dist-packages/tensorflow_probability/python/distributions/gaussian_process.py:312: UserWarning: Unable to detect statically whether the number of index_points is 1. As a result, defaulting to treating the marginal GP at `index_points` as a multivariate Gaussian. This makes some methods, like `cdf` unavailable.
  'Unable to detect statically whether the number of index_points is '


2602/2602 - 218s - loss: 1068.4477
Epoch 2/100
2602/2602 - 207s - loss: 279.4033
Epoch 3/100
2602/2602 - 206s - loss: 210.4949
Epoch 4/100
2602/2602 - 207s - loss: 186.4383
Epoch 5/100
2602/2602 - 207s - loss: 181.8416
Epoch 6/100
2602/2602 - 207s - loss: 174.3319
Epoch 7/100
2602/2602 - 206s - loss: 161.4740
Epoch 8/100
2602/2602 - 206s - loss: 155.2466
Epoch 9/100
2602/2602 - 206s - loss: 152.3885
Epoch 10/100
2602/2602 - 206s - loss: 150.1657
Epoch 11/100
2602/2602 - 206s - loss: 148.1109
Epoch 12/100
2602/2602 - 206s - loss: 146.4866
Epoch 13/100
2602/2602 - 206s - loss: 143.2140
Epoch 14/100
2602/2602 - 206s - loss: 137.1259
Epoch 15/100
2602/2602 - 206s - loss: 130.6922
Epoch 16/100
2602/2602 - 206s - loss: 122.6944
Epoch 17/100
2602/2602 - 205s - loss: 116.6692
Epoch 18/100
2602/2602 - 205s - loss: 112.5970
Epoch 19/100
2602/2602 - 205s - loss: 109.7081
Epoch 20/100
2602/2602 - 205s - loss: 107.9629
Epoch 21/100
2602/2602 - 205s - loss: 106.2153
Epoch 22/100
2602/2602 - 204s - l

In [ ]:
plt.plot(list(history.history.values())[0],'k-o')

In [ ]:
yhat5 = model5.predict(X_test)
model5.evaluate(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/tensorflow_probability/python/distributions/gaussian_process.py:312: UserWarning: Unable to detect statically whether the number of index_points is 1. As a result, defaulting to treating the marginal GP at `index_points` as a multivariate Gaussian. This makes some methods, like `cdf` unavailable.
  'Unable to detect statically whether the number of index_points is '
/usr/local/lib/python3.7/dist-packages/tensorflow_probability/python/distributions/gaussian_process.py:312: UserWarning: Unable to detect statically whether the number of index_points is 1. As a result, defaulting to treating the marginal GP at `index_points` as a multivariate Gaussian. This makes some methods, like `cdf` unavailable.
  'Unable to detect statically whether the number of index_points is '


1157/1157 [==============================] - 28s 23ms/step - loss: 23.3801


23.380134504215327

In [ ]:
from sklearn.metrics import max_error, mean_squared_error, explained_variance_score, mean_absolute_error, mean_squared_log_error, median_absolute_error, r2_score, mean_poisson_deviance, mean_gamma_deviance

In [ ]:
print('max_error',max_error(yhat5, y_test))
print('mean_squared_error',mean_squared_error(yhat5, y_test))
print('explained_variance_score', explained_variance_score(yhat5, y_test))
print('mean_absolute_error', mean_absolute_error(yhat5, y_test))
print('mean_squared_log_error', mean_squared_log_error(yhat5, y_test))
print('median_absolute_error', median_absolute_error(yhat5, y_test))
print('r2_score', r2_score(yhat5, y_test))
print('mean_poisson_deviance', mean_poisson_deviance(yhat5, y_test))
print('mean_gamma_deviance', mean_gamma_deviance(yhat5, y_test))

max_error 3.2991967891956167
mean_squared_error 0.16179387849704072
explained_variance_score 0.4704527629909756
mean_absolute_error 0.30325013620474256
mean_squared_log_error 0.001331694989095308
median_absolute_error 0.24453280908069708
r2_score 0.47012671621389845
mean_poisson_deviance 0.016078432304393535
mean_gamma_deviance 0.0016085843065658662


In [ ]:
plt.figure(figsize=[15, 10])  # inches
plt.plot(X, y, 'b.', label='observed');

num_samples = 7
for i in range(num_samples):
  sample_ = yhat5
  plt.plot(X_test,
           sample_[..., 0].T,
           'r',
           linewidth=0.9,
           label='ensemble means' if i == 0 else None);


ax=plt.gca();
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
ax.spines['left'].set_position(('data', 0))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#ax.spines['left'].set_smart_bounds(True)
#ax.spines['bottom'].set_smart_bounds(True)
plt.legend(loc='center left', fancybox=True, framealpha=0., bbox_to_anchor=(1.05, 0.5))
